# Ouray Defensible Space Analysis

Author: **Bryce A Young** | 
Created: **2024-12-05** | 
Modified: **2024-12-17**

In this notebook, we analyze fuel distributions within the defensible space of every building in Ouray County.

This analysis workflow also documents cleaning and preparing raw data for the analysis.

#### Data 
- (vector) Microsoft Building Footprints
- (raster) SILVIS Global WUI raster
- (raster) LiDAR-derived rasters

#### Workflow 
- Create HIZ boundaries around each building footprint
- Count number of homes within each HIZ
- Obtain WUI class for each home
- Get zonal summary values of each LiDAR-derived raster for each HIZ

## Step 0: Setup Environment
---

In [1]:
import os
### Directory ###
# Repository root
root = 'E:/_PROJECTS/Ouray_ParcelRisk/ouray'

### Data paths ###
# Folder where all the data inputs and outputs will live
data = 'E:/_PROJECTS/Ouray_ParcelRisk/data'
# Folder for all SILVIS WUI files
silvis_wui = os.path.join(data, 'silvis_wui')
# Folder for microsoft building footprints
mbf = os.path.join(data, 'building_footprints')
# Folder containing LiDAR-derived rasters
tiffs_from_las = os.path.join(data, 'tiffs_from_las')
# Folder countaining the Ouray County boundary
county_boundary = os.path.join(data, 'county_boundary')

In [2]:
os.chdir(root)
os.getcwd()

'E:\\_PROJECTS\\Ouray_ParcelRisk\\ouray'

## Step 1: Create HIZ Boundaries around each Structure
---
**Background: obtaining building footprints**  
*I searched "Ouray County Colorado Microsoft Building Footprints" and found a Colorado state website that had parsed the Microsoft Building Footprints dataset into county datasets for all of Colorado. So I was able to directly download the Ouray footprints without having to do any data manipulation myself.*

*After downloading the footprints, I added them to ArcGIS Pro and viewed them on top of my LiDAR data. Especially the `zentropy` layer shows buildings very well. I noticed that the footprint geometries are offset from the location of the buildings in the LiDAR rasters. So I used 'rubber sheeting' in ArcGIS Pro to put control points and target points that were intended to bring the building footprints closer to the buildings in the LiDAR rasters.*

*The resulting shapefile was saved to `ouray_footprints_rs.shp` where 'rs' denotes 'rubber sheet' transformation of the original footprint layer. Original footprints are saved in the same directory as `Ouray_County_Buildings.shp`.*

### Import geometries and run HIZ script

**TO DO**: I have to re-do this rubber sheeting and make sure to save the result as its own shapefile

In [ ]:
import geopandas as gpd

# Import building footprints
footprints = gpd.read_file(os.path.join(mbf, 'Ouray_County_Buildings.shp'))

In [ ]:
from utils.HIZ import get_hiz

# Create defensible space zones around all building footprints
hiz = get_hiz(footprints)
hiz.head() # Preview data

'e:\\_PROJECTS\\Ouray_ParcelRisk\\ouray\\workflows'

Now that we have created the HIZ boundaries around each building footprints, we need to save each one as its own geopackage for analysis. This could also be a shapefile, but a geopackage is more modern and generally preferred in 2025, so we'll stick with that.

Because of the difference in alignment between buildings in the LiDAR data and the outlines provided by Microsoft Building Footprints, we're actually going to use this first buffer as an analog for the building itself. This will help avoid incuding the building pixels from the raster in the analysis.

Below, we save each geometry column as its own geopackage, then we will restart the kernel to save memory and re-import the individual files.

**TO DO**: If I am using buffer_z1 as the actual footprint, then I'm going to want to rename these to keep everything straight.

In [ ]:
# Save each as its own geopackage

# In order to save a shapefile of HIZs, each geometry column needs to be saved as a separate geopackage
# Make sure each file contains one geometry column, and also 'FID' and "County'.
geom_cols = ['geometry', 'buffer_z1', 'buffer_z2', 'buffer_z3']
non_geom_cols = ['FID', 'County']

# Iterate and save each as a separate Shapefile
for geom_col in geom_cols:
    # Create a temporary GeoDataFrame with only the current geometry column and non-geometry columns
    temp_gdf = hiz[non_geom_cols + [geom_col]].copy()
    temp_gdf.set_geometry(geom_col, inplace=True)  # Loop through columns, setting each as the geometry column
    
    # Save to Shapefile
    temp_gdf.to_file(f'E:\_PROJECTS\Ouray_ParcelRisk\data\hiz_geoms\hiz_{geom_col}.gpkg', driver='GPKG')

In [ ]:
# Load HIZ geoms into environment

import geopandas as gpd

z1 = gpd.read_file("hiz_buffer_z1.gpkg")
z2 = gpd.read_file("hiz_buffer_z2.gpkg")
z3 = gpd.read_file("hiz_buffer_z3.gpkg")

## Step 2: Count the Number of Adjacent Structures
---

In order to obtain a proxy for structure density, we are going to count the number of adjacent structures in the vicinity of each home. We will append these counts to the geopackages.

In [ ]:
from utils.HIZ import structures_per_zone

buffer_cols = ['buffer_z1', 'buffer_z2', 'buffer_z3']  # Define your buffer zone columns
counts_df = structures_per_zone(gdf, 'footprints', buffer_cols)
counts_df.head()

**TO DO**: Does `counts_df` have geometries? Might need to add the extra step of appending that info to the footprint gdf. I think I ran this function in Gunnison so I should go look at those notebooks. 

## Step 3: Attributing WUI type to each home
---

#### Creating a WUI type raster for Ouray County
Right now I have the raw tiles from SILVIS GLobal WUI. Two tiles are required to cover the entire county area. I downloaded the entire North America dataset and found the correct tiles to cover the area. These are the raw data used here. 

First let's import the tiles and preview their metadata.

In [7]:
# Import Global WUI tiles
import rasterio as rio
import numpy as np
from utils.raster import read_raster

# Read in arrays and extract metadata
wui1 = os.path.join(silvis_wui, 'X0073_Y0064', 'wui.tif')
wui2 = os.path.join(silvis_wui, 'X0073_Y0065', 'wui.tif')

arr1, prof1 = read_raster(wui1, layer=1, profile=True)
arr2, prof2 = read_raster(wui2, layer=1, profile=True)

# Preview metadata
print('arr1: ', prof1)
print('arr2: ', prof2)

arr1:  {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 255.0, 'width': 10000, 'height': 10000, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Azimuthal_Equidistant",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Azimuthal_Equidistant"],PARAMETER["latitude_of_center",52],PARAMETER["longitude_of_center",-97.5],PARAMETER["false_easting",8264722.17686],PARAMETER["false_northing",4867518.35323],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(10.0, 0.0, 7300000.0,
       0.0, -10.0, 3500000.0), 'blockysize': 1, 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
arr2:  {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 255.0, 'width': 10000, 'height': 10000, 'count': 1, 'crs': CRS.from_wkt('PROJCS["Azimuthal_Equidistant",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,2

<Projected CRS: PROJCS["NAD83_Conus_Albers",GEOGCS["GCS_NAD83",DAT ...>
Name: NAD83_Conus_Albers
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

### Clip WUI rasters to Analysis Area (county boundary) - put this section in raster_prep.ipynb

Now we're going to clip the WUI rasters to the county boundary, setting values to nodata where county_boundary_raster == 0. The county boundary will be the snap raster and the source of truth for raster extent.

In [ ]:
# Reproject WUI rasters to same CRS as county boundary
# NOTE: make sure it's also the same CRS as the homes
import rasterio
from rasterio.mask import mask
from rasterio.warp import transform_geom
import geopandas as gpd
import numpy as np

# Paths to your files
raster_path = 'path/to/your/raster.tif'
vector_path = 'path/to/your/vector_boundary.shp'
output_path = 'path/to/output/clipped_raster.tif'

# Set desired nodata value
nodata_value = 255

# Step 1: Read vector boundary and reproject to raster CRS
vector = gpd.read_file(vector_path)

# Open the raster to get its CRS
with rasterio.open(raster_path) as src:
    raster_crs = src.crs

# Reproject vector boundary to match raster CRS
vector_reprojected = vector.to_crs(raster_crs)

# Step 2: Clip the raster using the reprojected vector geometry
geometries = [geom.__geo_interface__ for geom in vector_reprojected.geometry]

# Open raster and clip
with rasterio.open(raster_path) as src:
    # Perform the clipping
    clipped_raster, clipped_transform = mask(src, geometries, crop=True, nodata=nodata_value)
    
    # Update the metadata
    clipped_meta = src.meta.copy()
    clipped_meta.update({
        "driver": "GTiff",
        "height": clipped_raster.shape[1],
        "width": clipped_raster.shape[2],
        "transform": clipped_transform,
        "nodata": nodata_value
    })

# Step 3: Replace values outside the mask (optional but ensures consistency)
clipped_raster = np.where(clipped_raster == nodata_value, nodata_value, clipped_raster)

# Step 4: Save the clipped raster
with rasterio.open(output_path, 'w', **clipped_meta) as dst:
    dst.write(clipped_raster)

print(f"Clipped raster saved to: {output_path}")


In [ ]:
import numpy as np
from utils.raster import clip_raster

wui = clip_raster()

In [ ]:
# Save new WUI array to file
from utils.raster import write_raster

write_raster()

### Get WUI raster values per home

Now that we have the WUI rasters clipped and in the right CRS, we're going to find the value of the raster at the centriod of each home. Those values will be appended to the HIZ geodataframe.

In [ ]:
# Write a function to do get the centriod of each geometry and get the raster value at the centriod

In [ ]:
# Append values to gdf


In [ ]:
# Save gdf to file

## Step 4: Get canopy cover info for each HIZ
---

### Read in LiDAR-derived rasters

I created these rasters in R using the **lidR** package and methods documented in this repository under `r_workflows`.

In [6]:
# Import data
import rasterio as rio
import numpy as np
from utils.raster import read_raster

# Setup environment
# ----
# Path to all las-derived rasters
tiffs_from_las = r"E:/_PROJECTS/Ouray_ParcelRisk/data/tiffs_from_las"
# County, municipality etc. where the rasters cover
aoi = 'ouray'
# ----

# Initiate list to store objects
lidar_rasters = []
lidar_raster_profiles = []

# Canopy cover 0-2m
cc0_2, cc0_2_profile = read_raster(os.path.join(tiffs_from_las, '{0}_cc0_2m.tif'.format(aoi)), layer=1, profile=True)
lidar_rasters.append(cc0_2)
lidar_raster_profiles.append(cc0_2_profile)

# Canopy cover 2-4m
cc2_4, cc2_4_profile = read_raster(os.path.join(tiffs_from_las, '{0}_cc2_4m.tif'.format(aoi)), layer=1, profile=True)
lidar_rasters.append(cc2_4)
lidar_raster_profiles.append(cc2_4_profile)

# Canopy cover 4-8m
cc4_8, cc4_8_profile = read_raster(os.path.join(tiffs_from_las, '{0}_cc4_8m.tif'.format(aoi)), layer=1, profile=True)
lidar_rasters.append(cc4_8)
lidar_raster_profiles.append(cc4_8_profile)

# Canopy cover 8-40m
cc8_40, cc8_40_profile = read_raster(os.path.join(tiffs_from_las, '{0}_cc8_40m.tif'.format(aoi)), layer=1, profile=True)
lidar_rasters.append(cc8_40)
lidar_raster_profiles.append(cc8_40_profile)

# Canopy height model
chm, chm_profile = read_raster(os.path.join(tiffs_from_las, '{0}_chm.tif'.format(aoi)), layer=1, profile=True)
lidar_rasters.append(chm)
lidar_raster_profiles.append(chm_profile)

# Ladder fuels
ladder, ladder_profile = read_raster(os.path.join(tiffs_from_las, '{0}_ladderfuels.tif'.format(aoi)), layer=1, profile=True)
lidar_rasters.append(ladder)
lidar_raster_profiles.append(ladder_profile)

# Point density
density, density_profile = read_raster(os.path.join(tiffs_from_las, '{0}_pointDensity.tif'.format(aoi)), layer=1, profile=True)
lidar_rasters.append(density)
lidar_raster_profiles.append(density_profile)

# Z-Entropy
zentropy, zentropy_profile = read_raster(os.path.join(tiffs_from_las, '{0}_zentropy.tif'.format(aoi)), layer=1, profile=True)
lidar_rasters.append(zentropy)
lidar_raster_profiles.append(zentropy_profile)

# Print metadata for quality inspection
print('0-2m: ', cc0_2_profile)
print('2-4m: ', cc2_4_profile)
print('4-8m: ', cc4_8_profile)
print('8+m: ', cc8_40_profile)
print('chm: ', chm_profile)
print('ladder: ', ladder_profile)
print('density: ', density_profile)
print('zentropy: ', zentropy_profile)

0-2m:  {'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 42881, 'height': 47000, 'count': 1, 'crs': CRS.from_wkt('LOCAL_CS["NAD83(2011) / UTM zone 13N",UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(1.0, 0.0, 236950.0,
       0.0, -1.0, 4247000.0), 'blockysize': 1, 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
2-4m:  {'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 42881, 'height': 47000, 'count': 1, 'crs': CRS.from_wkt('LOCAL_CS["NAD83(2011) / UTM zone 13N",UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(1.0, 0.0, 236950.0,
       0.0, -1.0, 4247000.0), 'blockysize': 1, 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
4-8m:  {'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 42881, 'height': 47000, 'count': 1, 'crs': CRS.from_wkt('LOCAL_CS["NAD83(2011) / UTM zone 13N",UNIT["metre",1,AUTHORITY["EPSG","900

### put this section in raster_prep.ipynb!

The metadata of these rasters needs updating. For instance, we can change the dtype on many of these rasters to save space and processing speed. To do that, we will have to solve for the nodata value as nan, since that requires a float dtype. Ladder fuels are a binary raster.

We want the CRS and shape of these rasters to match the CRS and shape of the buildings and the county raster. So let's do some raster cleanup and make these changes accordingly.

In [ ]:
# Code as necessary

Now let's plot the rasters to see what they look like

In [ ]:
# Plot all 4 rasters in 2x2 subplots
import matplotlib.pyplot as plt

# Code

Notice the missing pixel values seen in the plot. I fill these in with a 3x3 moving window that fills in missing values with the average value of the 8 surrounding pixels.

Creating a window required a few steps. First, defining what the window does. I call this function `fill_nan`. I use `generic_filter` from the `scipy` library, which does this automatically, providing three arguments: the **array** to filter, the **function** of the window, and the **size** of the window. Optionally, we can set **mode** based on how we want to treat pixels outside of the array. Let's stick with the default *reflect*. You can read more about options here: [https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.generic_filter.html]

### Fill in missing values - put this section in raster_prep.ipynb

#### Canopy Cover 0-2m

**TO DO:** This code needs cleaned up so that it uses the class I created, or maybe I just need to simplify things by breaking down the class into a function.

In [ ]:
# Fill NaN values with moving window
from scipy.ndimage import generic_filter
from utils.raster import FillNaN # NOTE: am I importing this class correctly?

# Apply the moving window function to fill missing values in cc0_2
window_size = (3, 3)
arr_max = generic_filter(arr_hmax, fill_nan, size=window_size, mode='reflect')

In [ ]:
# Preview to ensure quality
import matplotlib.pyplot as plt

plt.imshow(cc0_2)

In [ ]:
# Save array, overwriting
from utils.raster import write_raster

write_raster('file/path/raster.tif')

#### Canopy Cover 2-4m

In [ ]:
# Fill NaN values with moving window
from scipy.ndimage import generic_filter
from utils.raster import FillNaN # NOTE: am I importing this class correctly?

# Apply the moving window function to fill missing values in cc2_4
window_size = (3, 3)
arr_max = generic_filter(arr_hmax, fill_nan, size=window_size, mode='reflect')

In [ ]:
# Preview to ensure quality
import matplotlib.pyplot as plt

plt.imshow(cc2_4)

In [ ]:
# Save array, overwriting
from utils.raster import write_raster

write_raster('file/path/raster.tif')

#### Canopy Cover 4-8m

In [ ]:
# Fill NaN values with moving window
from scipy.ndimage import generic_filter
from utils.raster import FillNaN # NOTE: am I importing this class correctly?

# Apply the moving window function to fill missing values in cc4_8
window_size = (3, 3)
arr_max = generic_filter(arr_hmax, fill_nan, size=window_size, mode='reflect')

In [ ]:
# Preview to ensure quality
import matplotlib.pyplot as plt

plt.imshow(cc4_8)

In [ ]:
# Save array, overwriting
from utils.raster import write_raster

write_raster('file/path/raster.tif')

#### Canopy Cover 8m+

In [ ]:
# Fill NaN values with moving window
from scipy.ndimage import generic_filter
from utils.raster import FillNaN # NOTE: am I importing this class correctly?

# Apply the moving window function to fill missing values in cc_8
window_size = (3, 3)
arr_max = generic_filter(arr_hmax, fill_nan, size=window_size, mode='reflect')

In [ ]:
# Preview to ensure quality
import matplotlib.pyplot as plt

plt.imshow(cc_8)

In [ ]:
# Save array, overwriting
from utils.raster import write_raster

write_raster('file/path/raster.tif')

#### Get raster values per HIZ across the county

Recall the geodataframes of home ignition zones that we created at the beginning of this workflow. It contains geometries for **XX** building footprins and their home ignition zones. We're going to look at the vegetation within each HIZ to get an idea of the vegetation profile around each home.

For the purposes of reducing wildfire risk to individual homes, it's important that we analyze raster values within each buffer zone. GeoPandas comes in handy for this task.

Our workflow for this calculation - written in plain English - is as follows:
1. **Mask raster with geometries**: Recall that each home ignition zone gdf is a geoseries of Shapely geometries. We need to iterate through **each** of these **XX** objects and use them to mask our raster, object by object.
2. **Calculate average raster values within each HIZ**: once the mask is applied to only include a single building buffer, we'll take the average of all the pixels in the area. 
3. **Store respective canopy cover value per HIZ as a new column in each HIZ dataframe:** The average values will be stored as columns and each value will correspond to the geometry in the same row.

**TO DO:** decide the best way to organize these. I'm thinking column titles will be 'ccvalue_hiz' and will be appended to the individual HIZ shapefiles rather than the grandfather gdf.

THIS IS IN `ndvi_hiz.ipynb`.

In [ ]:
# Code for analysis

In [ ]:
# Calculate average raster value per HIZ
def calculate_avg(arr):
    
    # Count non-NaN values in the array
    non_nan_count = np.sum(~np.isnan(arr))
    # Compute the sum of non-NaN values
    non_nan_sum = np.sum(arr[~np.isnan(arr)])
    # Calculate the average (handle case when non_nan_count is 0 to avoid division by zero)
    avg = non_nan_sum / non_nan_count if non_nan_count > 0 else np.nan

    return avg

#### Append values to HIZ geopackages

In [ ]:
# Code to append values

In [3]:
# Save updated GDFs

## Discussion and Conclusions
---

- Broader picture of wildfire risk to communities and susceptibility and defensible space
- Describe the project in its entirety, in brief
- Here's how this notebook/workflow fits into the larger project
- Here's what we did.
- Here were the most difficult parts.
- Here were the key parts to get right.
- Time estimates for running the code.
- Information on computational expense.
- Room for improvement

## NOTES: Possibble to use this code somewhere?
---

We're going to clip our rasters to the extent of the given geometry for each of these.

Our simplified proxy for NFPA compliance is going to be the **average max height** of vegetation pixels within each zone.

The process of clipping rasters to geometries requires a fe steps. First, we're going to mask the max height raster `arr_max` with each HIZ geometry - `z1`, `z2`, and `z3`. To do this, we have to rasterize each geometry. The following 3 code blocks accomplish that, and save the masked raster outputs to arrays.

In [83]:
from rasterio.mask import mask
from rasterio.io import MemoryFile

# Prepare geometries from GeoDataFrame for masking
geoms = z1['geometry'].values # Zone 1

# Create a new in-memory dataset
with MemoryFile() as memfile:
    # Define metadata based on the properties of arr_max
    meta = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'count': 1,
        'width': arr_max.shape[1],
        'height': arr_max.shape[0],
        'crs': hmax.crs,  # Update with the correct CRS as necessary
        'transform': hmax.transform,  # Update with the correct transform if available
        'nodata': np.nan
    }

    with memfile.open(**meta) as hmax_dataset:
        # Write arr_max to the in-memory dataset
        hmax_dataset.write(arr_max, 1)

        # Apply the mask using the geometries
        z1_out_values, z1_out_transform = mask(hmax_dataset, geoms, crop=True, nodata=np.nan)


In [84]:
geoms = z2['geometry'].values # Zone 2

# Create a new in-memory dataset
with MemoryFile() as memfile:
    # Define metadata based on the properties of arr_max
    meta = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'count': 1,
        'width': arr_max.shape[1],
        'height': arr_max.shape[0],
        'crs': hmax.crs,  # Update with the correct CRS as necessary
        'transform': hmax.transform,  # Update with the correct transform if available
        'nodata': np.nan
    }

    with memfile.open(**meta) as hmax_dataset:
        # Write arr_max to the in-memory dataset
        hmax_dataset.write(arr_max, 1)

        # Apply the mask using the geometries
        z2_out_values, z2_out_transform = mask(hmax_dataset, geoms, crop=True, nodata=np.nan)


In [85]:
geoms = z3['geometry'].values # Zone 3

# Create a new in-memory dataset
with MemoryFile() as memfile:
    # Define metadata based on the properties of arr_max
    meta = {
        'driver': 'GTiff',
        'dtype': 'float32',
        'count': 1,
        'width': arr_max.shape[1],
        'height': arr_max.shape[0],
        'crs': hmax.crs,  # Update with the correct CRS as necessary
        'transform': hmax.transform,  # Update with the correct transform if available
        'nodata': np.nan
    }

    with memfile.open(**meta) as hmax_dataset:
        # Write arr_max to the in-memory dataset
        hmax_dataset.write(arr_max, 1)

        # Apply the mask using the geometries
        z3_out_values, z3_out_transform = mask(hmax_dataset, geoms, crop=True, nodata=np.nan)


Next, let's define a function for computing the average value of all the pixels.

We know that our zones are all donut-shaped. If we divide the sum of the pixels by the size of the array, then pixels both inside and outside the donut will be included in the size, and that will throw off our calculation. The above 3 code blocks have set all the values outside and inside of the donuts to `NaN`, so now we make sure our `calculate_avg` function divided by the number of *non-NaN* pixels in the array.

### RESULTS
All that's left to do is calculate the average max height per zone. Here are the results:

In [87]:
calculate_avg(z3_out_values)

2.3484971919686135

In [88]:
calculate_avg(z2_out_values)

3.054104777520576

In [89]:
calculate_avg(z1_out_values)

3.9157764165088382

### CONCLUSIONS

There are some insights and some caveats to draw from this from a fire mitigation perspective.

For a NFPA-compliant home ignition zone, we would expect to see taller average height values further away from the home. Instead, we are seeing the reverse.

The Z1 values have possibly been skewed by the home itself. Since the Microsoft building footprint does not perfectly align with the location of the building in the point cloud, it is likely that the building itself has been included in the max height calculation. A way to handle this would be to classify the point cloud to include buildings, and derive the building footprint directly from the LiDAR data.

The Z2 and Z3 values are probably more reliable. It tells us that there is tall vegetation surrounding the home. If the average height exceeds 3m in Z2, it is likely that this property could benefit from removing some trees and ensuring that the ground is clear of continuous flammable fuels. Since Z3 average max height is lower, it's likely that Z3 will not carry fire, although the average max height could be skewed by clumps of tall, continuous forest fuels interspersed with grassland, which is typical of this environment.

My conclusion is that average max height does not produce an actionable insight for home risk reduction. There is potential to develop better ways of producing actionable insights for home wildfire risk mitigation, but none have been developed (based on a meticulous review of over 150 papers and models). While the results presented here may prove to be a useful predictor in a random forest or multi-layer perceptron model, I will continue to test the value of different LiDAR metrics using the framework developed through this project.